In [1]:
from datasets_dgl.data_dgl import *
from  easydict  import EasyDict
from datasets_dgl.utils import to_scipy

torch.cuda.set_device(0)  # 指第几块GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# pubmed 攻击用的是meta_self所以攻击test多
dataset_name = 'Attack-Cora'
attack       = 'Meta_Self-0.25'

################# no attack
DATASET0 = EasyDict()
DATASET0.ATTACK = {
    "data"          :dataset_name,
    "attack"        :attack.split('-')[0],
    "ptb_rate"      :attack.split('-')[1],
    "train_size"    :0.9,
    "val_size"      :0.05,
    "test_size"     :0.05,
    "group"         :9,      # train_size * 10
    "use_g1_split"  :False
}
dataset  = load_attack_data(DATASET0['ATTACK'])
graph = dataset.graph
graph = graph.to(device)

/home/songsh/anaconda3/envs/ALOHA/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[I] Loading dataset Attack-Cora...
train_mask, test_mask, val_mask sizes : tensor(2235) tensor(125) tensor(125)
[I] Finished loading.
[I] Data load time: 1.0589s


In [2]:
features = graph.ndata["feat"]
labels = graph.ndata["label"]
masks = graph.ndata["train_mask"], graph.ndata["val_mask"], graph.ndata["test_mask"]

In [3]:
import dgl
import dgl.nn as dglnn

import torch
import torch.nn as nn
import torch.nn.functional as F

class GCN(nn.Module):
    def __init__(self, in_size, hid_size, out_size):
        super().__init__()
        self.layers = nn.ModuleList()
        # two-layer GCN
        self.layers.append(
            dglnn.GraphConv(in_size, hid_size, activation=F.relu)
        )
        self.layers.append(dglnn.GraphConv(hid_size, out_size))
        self.dropout = nn.Dropout(0.5)

    def forward(self, g, features):
        h = features
        for i, layer in enumerate(self.layers):
            if i != 0:
                h = self.dropout(h)
            h = layer(g, h)
        return h


def evaluate(g, features, labels, mask, model):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)


def train(g, features, labels, masks, model):
    # define train/val samples, loss function and optimizer
    train_mask = masks[0]
    val_mask = masks[1]
    loss_fcn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=5e-4)

    # training loop
    for epoch in range(200):
        model.train()
        logits = model(g, features)
        loss = loss_fcn(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = evaluate(g, features, labels, val_mask, model)
        print(
            "Epoch {:05d} | Loss {:.4f} | Accuracy {:.4f} ".format(
                epoch, loss.item(), acc
            )
        )

In [4]:
# create GCN model
in_size = features.shape[1]
out_size = dataset.num_classes
model = GCN(in_size, 16, out_size).to(device)

# model training
print("Training...")
train(graph, features, labels, masks, model)

# test the model
print("Testing...")
acc = evaluate(graph, features, labels, masks[2], model)
print("Test accuracy {:.4f}".format(acc))

Training...
Epoch 00000 | Loss 1.9477 | Accuracy 0.2720 
Epoch 00001 | Loss 1.8362 | Accuracy 0.3680 
Epoch 00002 | Loss 1.7498 | Accuracy 0.3840 
Epoch 00003 | Loss 1.6621 | Accuracy 0.4240 
Epoch 00004 | Loss 1.5761 | Accuracy 0.4320 
Epoch 00005 | Loss 1.5038 | Accuracy 0.4720 
Epoch 00006 | Loss 1.4351 | Accuracy 0.5440 
Epoch 00007 | Loss 1.3470 | Accuracy 0.5760 
Epoch 00008 | Loss 1.2884 | Accuracy 0.6240 
Epoch 00009 | Loss 1.2050 | Accuracy 0.6800 
Epoch 00010 | Loss 1.1292 | Accuracy 0.6880 
Epoch 00011 | Loss 1.0637 | Accuracy 0.6960 
Epoch 00012 | Loss 0.9977 | Accuracy 0.7040 
Epoch 00013 | Loss 0.9574 | Accuracy 0.7200 
Epoch 00014 | Loss 0.8835 | Accuracy 0.7280 
Epoch 00015 | Loss 0.8502 | Accuracy 0.7280 
Epoch 00016 | Loss 0.7830 | Accuracy 0.7360 
Epoch 00017 | Loss 0.7410 | Accuracy 0.7360 
Epoch 00018 | Loss 0.7085 | Accuracy 0.7360 
Epoch 00019 | Loss 0.6672 | Accuracy 0.7360 
Epoch 00020 | Loss 0.6026 | Accuracy 0.7360 
Epoch 00021 | Loss 0.5743 | Accuracy 0.7360